In [6]:
import os

In [7]:
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\MLOPS\\datascienceproject\\research'

In [8]:
os.chdir("../")
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\MLOPS\\datascienceproject'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [11]:
class ConfigurationManager:
    def __init__ (self,
                  config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
    
    
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        
    

In [12]:
import os
import urllib.request  as request
from src.datascience import logger
import zipfile

In [13]:
#data ingestion component
import requests
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
        
        #downlaoding the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
                
            )
            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists ")
            
    def extract_zip_file(self):
        """
        zip_file_path:str
        extracts the zip file into the data directory
        function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
            
        
        
        


In [14]:
try:
    config= ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2025-07-31 13:36:49,098: common: yaml file:config\config.yaml loaded successfully]
[2025-07-31 13:36:49,132: common: yaml file:params.yaml loaded successfully]
[2025-07-31 13:36:49,153: common: yaml file:schema.yaml loaded successfully]
[2025-07-31 13:36:49,159: common: created directory at: artifacts_root]
[2025-07-31 13:36:49,165: common: created directory at: artifacts/data_ingestion]
[2025-07-31 13:36:51,178: 2867500463: artifacts/data_ingestion/data.zip downloaded! with the following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8884:1F55B0:889C9:1840AB:688B241B
Accept-Ranges: bytes
Date: Thu, 31 Jul 2025 08:06:51 GMT
Via: 1